In [10]:
from functools import reduce
from operator import itemgetter, mul
from itertools import groupby
from sympy.ntheory import factorint, totient
from math import gcd

from itertools import chain, combinations

def product(S):
    return reduce(mul, S)

In [11]:
def coprime_and_divisible_by_3(n):
    if n%3 == 0:
        return 0
    else:
        prime_factors = factorint(n)
        primes = list(prime_factors.keys())
        prime_combinations = chain(
            (r, s) for r in range(1, len(primes)+1) for s in combinations(primes, r)
        )

        num_multiples_of_3 = (
            (r, n//(3*product(sub_primes)))
            for r, sub_primes in prime_combinations
        )

        num_not_coprime_and_divisible_by_3 = sum(
            ((-1)**((r+1)%2))*size_subset
            for r, size_subset in num_multiples_of_3
        )

        return (n//3) - num_not_coprime_and_divisible_by_3

In [12]:
def coprime_and_divisible_by_3_test(n):
    return sum((gcd(n, i) == 1) and (i%3 ==0) for i in range(1, n+1))

In [13]:
all(coprime_and_divisible_by_3(n) == coprime_and_divisible_by_3_test(n) for n in range(1, 1001))

True

In [32]:
def f(n):
    if n%3 == 0:
        return coprime_and_divisible_by_3(n)
    else:
        return (totient(n) - 2*coprime_and_divisible_by_3(n))

In [40]:
def f_test(n):
    return sum((gcd(n, i) == 1) and ((i+n)%3 ==0) for i in range(1, n+1))

In [33]:
import pandas as pd

In [50]:
data = [
    (n, 2*n - 3, totient(n), coprime_and_divisible_by_3_test(n), f(n),  f_test(n))
    for n in range(2, 31)
]

df = pd.DataFrame(data, columns=['n', 'num_reflections', 'euler totient', 'coprime and divisible by 3', 'f', 'f_test'])

In [51]:
all(f(n) == f_test(n) for n in range(2, 1001))

True

In [52]:
df

,n,num_reflections,euler totient,coprime and divisible by 3,f,f_test
0,2,1,1,0,1,1
1,3,3,2,0,0,0
2,4,5,2,1,0,0
3,5,7,4,1,2,2
4,6,9,2,0,0,0
5,7,11,6,2,2,2
6,8,13,4,1,2,2
7,9,15,6,0,0,0
8,10,17,4,2,0,0
9,11,19,10,3,4,4


In [27]:
[f(n) for n in range(1, 10)]

[0.5, 0.5, 0, 0.5, 1.5, 0, 2.0, 1.5, 0]

In [55]:
def num_rays(n):
    if n%2 == 0:
        return 0
    else:
        return f((n+3)//2)

In [56]:
[num_rays(n) for n in range(1, 10)]

[1, 0, 0, 0, 0, 0, 2, 0, 0]

In [57]:
[num_rays(n) for n in range(1000000-10, 1000000+10)]

[0,
 159412,
 0,
 0,
 0,
 160020,
 0,
 66668,
 0,
 0,
 0,
 80840,
 0,
 142856,
 0,
 0,
 0,
 121200,
 0,
 76920]

In [61]:
num_rays(1000001)

80840

In [60]:
num_rays(12017639147)

1209002624

In [55]:
[
    coprime_and_divisible_by_3(n)
    for n in range(20)
]

[0, 0, 0, 0, 1, 1, 0, 2, 1, 0, 2, 3, 0, 4, 2, 0, 3, 5, 0, 6]

# Old

In [1]:
def euler_totient_prime_mod_3(p, a):
    if (p % 3) == 1:
        res = p**(a-1)*(p - 1)//3
        return (res, res, res)
    elif (p % 3) == 2:
        x = p**(a-1)*(p - 1)
        if a % 2 == 0:
            return ((x + 1)//3, (x + 1)//3, (x-2)//3)
        else:
            return ((x-1)//3, (x + 2)//3, (x-1)//3)
    else:
        raise ValueError(f"p={p} divisble by 3")

In [2]:
euler_totient_prime_mod_3(5, 2)

(7, 7, 6)

In [21]:
def combine_coprime_mod_3_solutions(p1, p2):
    prime1, c1 = p1
    prime2, c2 = p2
    r1 = prime1%3
    r2 = prime2%3
    new_cs = (
        (x1*x2, (r1*i1+r2*i2)%3)
        for i1, x1 in enumerate(c1)
        for i2, x2 in enumerate(c2)
    )
    new_cs = sorted(new_cs, key=itemgetter(1))
    new_cs = groupby(new_cs, key=itemgetter(1))
    new_cs = [
        sum(x for x, _ in l)
        for _, l in new_cs
    ]
    
    new_r = (r1*r2)%3
    
    return (new_r, new_cs)

In [23]:
def euler_totient_mod_3(n):
    prime_factors = factorint(n)
    prime_factors = sorted(list(prime_factors.items()), key=lambda x: x[0])
    print(prime_factors)
    prime_totients = (
        (((p%3)**a)%3, euler_totient_prime_mod_3(p, a)) for p, a in prime_factors
    )
    
    final_totients = reduce(combine_coprime_mod_3_solutions, prime_totients)
    
    return final_totients

In [26]:
euler_totient_mod_3(10)

[(2, 1), (5, 1)]


(1, [1, 2, 1])